# Code to flatten array and struct to lowest level

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [2]:
#Json taken from Internet for parsing
!cat "nested_json.json"

 {
    "medications":[{
            "aceInhibitors":[{
                "name":"lisinopril",
                "strength":"10 mg Tab",
                "dose":"1 tab",
                "route":"PO",
                "sig":"daily",
                "pillCount":"#90",
                "refills":"Refill 3"
            }],
            "antianginal":[{
                "name":"nitroglycerin",
                "strength":"0.4 mg Sublingual Tab",
                "dose":"1 tab",
                "route":"SL",
                "sig":"q15min PRN",
                "pillCount":"#30",
                "refills":"Refill 1"
            }],
            "anticoagulants":[{
                "name":"warfarin sodium",
                "strength":"3 mg Tab",
                "dose":"1 tab",
                "route":"PO",
                "sig":"daily",
                "pillCount":"#90",
                "refills":"Refill 3"
            }],
            "betaBlocker":[{
                "name":"metoprolol tartrate",
           

In [3]:
spark=SparkSession.builder.getOrCreate()

In [4]:
s=spark.read.json("nested_json.json",multiLine=True)

In [5]:
#making necessary imports

opt=[]
opt.append(1)
def flatten_df_full(nested_df):
    #print("is it")
    flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct' and c[1][:5] !="array"]
    #print("flat_cols : ",flat_cols )
    nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']
    array_cols=[c[0] for c in nested_df.dtypes if c[1][:5] == 'array']
    #print("array_cols",array_cols)
    #print("nested_cols :",nested_cols)
    for ac in array_cols:
        nested_df=nested_df.select(flat_cols+nested_cols+[x for x in array_cols if x <> ac ]+[F.explode(ac).alias(ac)])
        #print(nested_df.columns)
    nested_df = nested_df.select(flat_cols +array_cols+
                               [F.col(nc+'.'+c).alias(nc+'_'+c)
                                for nc in nested_cols
                                for c in nested_df.select(nc+'.*').columns])
    opt.pop()
    opt.append(nested_df)
    #print("hello ",",".join([x[1][:6] for x in nested_df.dtypes]))
    new_nested=[c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']
    new_array=[c[0] for c in nested_df.dtypes if c[1][:5] == 'array']
    #print("new_flat",new_flat)
    if new_nested or new_array :
        print("entering",new_nested,new_array)
        flatten_df_full(nested_df)
flatten_df_full(s)
opt_df=opt[0]

('entering', ['labs', 'medications', 'imaging_src_struc2t'], ['imaging_src_imaging'])
('entering', ['imaging_src_imaging'], ['medications_aceInhibitors', 'medications_antianginal', 'medications_anticoagulants', 'medications_betaBlocker', 'medications_diuretic', 'medications_mineral'])
('entering', ['medications_aceInhibitors', 'medications_antianginal', 'medications_anticoagulants', 'medications_betaBlocker', 'medications_diuretic', 'medications_mineral'], [])


In [6]:
#OPT_columns after flattening array and struct
len(opt_df.columns)

51

In [12]:
opt_df_pandas=opt_df.toPandas()

In [9]:
import re

In [19]:
opt_df_pandas.drop_duplicates(inplace=True)

In [18]:
opt_df_pandas.shape

(36, 51)

In [21]:
opt_df_pandas.head(5).T

,0,3,6,9,12
imaging_src_test,time,time,time,time,time
labs_location,Main Hospital Lab,Primary Care Clinic,Primary Care Clinic,Primary Care Clinic,Primary Care Clinic
labs_name,Arterial Blood Gas,BMP,BNP,BUN,Cardiac Enzymes
labs_time,Today,Today,3 Weeks,1 Year,Today
imaging_src_struc2t_age,25,25,25,25,25
imaging_src_struc2t_id,int,int,int,int,int
imaging_src_imaging_location,Main Hospital Radiology,Main Hospital Radiology,Main Hospital Radiology,Main Hospital Radiology,Main Hospital Radiology
imaging_src_imaging_name,Chest X-Ray,Chest X-Ray,Chest X-Ray,Chest X-Ray,Chest X-Ray
imaging_src_imaging_time,Today,Today,Today,Today,Today
medications_aceInhibitors_dose,1 tab,1 tab,1 tab,1 tab,1 tab


In [22]:
opt_df.dtypes

[('imaging_src_test', 'string'),
 ('labs_location', 'string'),
 ('labs_name', 'string'),
 ('labs_time', 'string'),
 ('imaging_src_struc2t_age', 'string'),
 ('imaging_src_struc2t_id', 'string'),
 ('imaging_src_imaging_location', 'string'),
 ('imaging_src_imaging_name', 'string'),
 ('imaging_src_imaging_time', 'string'),
 ('medications_aceInhibitors_dose', 'string'),
 ('medications_aceInhibitors_name', 'string'),
 ('medications_aceInhibitors_pillCount', 'string'),
 ('medications_aceInhibitors_refills', 'string'),
 ('medications_aceInhibitors_route', 'string'),
 ('medications_aceInhibitors_sig', 'string'),
 ('medications_aceInhibitors_strength', 'string'),
 ('medications_antianginal_dose', 'string'),
 ('medications_antianginal_name', 'string'),
 ('medications_antianginal_pillCount', 'string'),
 ('medications_antianginal_refills', 'string'),
 ('medications_antianginal_route', 'string'),
 ('medications_antianginal_sig', 'string'),
 ('medications_antianginal_strength', 'string'),
 ('medicati